### Import Libraries
#### Please use pip install pyamaze before running this code if not already executed

In [1]:
from pyamaze import maze, agent, COLOR, textLabel
import tracemalloc as memory_trace
import time
from IPython.display import display
import pandas as pd
from queue import PriorityQueue
import numpy as np

### Impelement A* Algortihm Functionality

In [2]:
class A_Star_Search : 
    
    def __init__(self, maze_size) : 
        self.maze_size = maze_size
    
    def load_maze(self) : 
        m = maze()
        maze_name = 'Maze_' + str(self.maze_size) + 'X' + str(self.maze_size)
        m.CreateMaze(loadMaze = maze_name + '.csv')
        return m
    
    def start_memory_tracing(self) :
        memory_trace.stop()
        memory_trace.start()
        
    def stop_memory_tracing(self) : 
        memory_size, memory_peak = memory_trace.get_traced_memory()
        return memory_size, memory_peak
    
    def initialize_maze(self) : 
        self.maze = self.load_maze()
        self.goal_node = self.maze._goal
        self.start_node = (self.maze_size, self.maze_size)
        
    def initialise_cost(self) : 
        self.next_node_cost = {node : 0 if node == self.start_node else float('inf') for node in self.maze.grid}
        self.total_cost = {node :  0 + self.get_euclidian_distance_heuristic_cost(self.start_node) if node == self.start_node else float('inf') for node in self.maze.grid}
        
    
    def execute_a_star_search(self):
        self.initialize_maze()
        self.initialise_cost()
        priority_queue = PriorityQueue()
        priority_queue.put((0 + self.get_euclidian_distance_heuristic_cost(self.start_node), self.get_euclidian_distance_heuristic_cost(self.start_node), self.start_node))
        
        explored_nodes = []
        path_traversed = {}
        
        start_time = time.time() * 1000
        self.start_memory_tracing()
        while not priority_queue.empty() : 
            
            current_node = priority_queue.get()[2]
            explored_nodes.append(current_node)
            
            if current_node == self.goal_node:
                break
                
            for __direction__ in ['N', 'S', 'E', 'W']:
                
                if self.maze.maze_map[current_node][__direction__] == 1 :
                    
                    if __direction__ == 'N' : 
                        next_node = (current_node[0] - 1, current_node[1])
                    
                    elif __direction__ == 'S' : 
                        next_node = (current_node[0] + 1, current_node[1])
                    
                    elif __direction__ == 'E' : 
                        next_node = (current_node[0], current_node[1] + 1)
                    
                    elif __direction__ == 'W' : 
                        next_node = (current_node[0], current_node[1] - 1)
                    
                    var_next_node_cost = self.next_node_cost[current_node] + 1
                    var_total_cost = var_next_node_cost + self.get_euclidian_distance_heuristic_cost(next_node)
                    
                    if var_total_cost < self.total_cost[next_node] : 
                        self.total_cost[next_node] = var_total_cost
                        self.next_node_cost[next_node] = var_next_node_cost  
                        priority_queue.put((var_total_cost, self.get_euclidian_distance_heuristic_cost(next_node), next_node))
                        path_traversed[next_node] = current_node
        
        end_time = time.time() * 1000
        time_taken = (end_time - start_time)
        
        memory_size, memory_peak = self.stop_memory_tracing()
        memory_consumed = round((memory_peak/(1024*1024)), 3)
       
        goal_nodes = self.find_goal_nodes(path_traversed, self.start_node, self.goal_node)
        
        statistics_df = pd.DataFrame(columns=['Maze Size', 'Time Taken (in ms)', 'Memory Consumed (in MB)', 'Number of Cells Explored', 'Number of Cell in Shortest Path to Goal'])
        statistics_dict = {}
        statistics_dict['Maze Size'] = str(self.maze_size) + 'X' + str(self.maze_size)
        statistics_dict['Time Taken (in ms)'] = time_taken
        statistics_dict['Memory Consumed (in MB)'] = memory_consumed
        statistics_dict['Number of Cells Explored'] = len(path_traversed) + 1
        statistics_dict['Number of Cell in Shortest Path to Goal'] = len(goal_nodes) + 1
        
        self.display_astar_path(explored_nodes, goal_nodes, time_taken, memory_consumed, len(path_traversed) + 1, len(goal_nodes) + 1)
        
        statistics_df = statistics_df.append(statistics_dict, ignore_index = True)
        
        return statistics_df
        
    def get_euclidian_distance_heuristic_cost(self, node):
        x, y = node
        goal_x, goal_y = self.maze._goal
        return np.sqrt(pow((goal_x - x), 2) + pow((goal_y - y), 2)) 
    
    def find_goal_nodes(self, path_traversed, start_node, goal_node) : 
        goal_nodes = {}

        while goal_node != start_node : 
            goal_nodes[path_traversed[goal_node]] = goal_node
            goal_node = path_traversed[goal_node]

        return goal_nodes

    def display_astar_path(self, explored_nodes, goal_nodes, time_taken, memory_consumed, len_path_traversed, len_goal_nodes) : 
        explored_path = agent(self.maze, x = self.maze_size, y = self.maze_size, goal = (1, 1), footprints = True, color=COLOR.red, filled = True)
        goal_path = agent(self.maze, x = self.maze_size, y = self.maze_size, footprints = True, color=COLOR.cyan)
        
        self.maze.tracePath({explored_path : explored_nodes}, delay = 10)
        self.maze.tracePath({goal_path : goal_nodes}, delay = 100)
        
        textLabel(self.maze, 'Maze Size ', str(self.maze_size) + 'X' + str(self.maze_size))
        textLabel(self.maze, 'Time Taken (in ms) ', time_taken)
        textLabel(self.maze, 'Memory Consumed (in MB) ', memory_consumed)
        textLabel(self.maze, 'Number of Cells Explored ', len_path_traversed)
        textLabel(self.maze, 'Number of Cell in Shortest Path to Goal ', len_goal_nodes)
        
        self.maze.run()
  

### Executing A* for Maze Size 20 X 20

In [6]:
astar_20 = A_Star_Search(20)

statistics = astar_20.execute_a_star_search()

statistics = statistics.style.applymap(lambda x:'white-space:nowrap')
display(statistics)


,Maze Size,Time Taken (in ms),Memory Consumed (in MB),Number of Cells Explored,Number of Cell in Shortest Path to Goal
0,20X20,8.975586,0.037000,359,65


### Executing A* for Maze Size 30 X 30

In [7]:
astar_30 = A_Star_Search(30)

statistics = astar_30.execute_a_star_search()

statistics = statistics.style.applymap(lambda x:'white-space:nowrap')
display(statistics)


,Maze Size,Time Taken (in ms),Memory Consumed (in MB),Number of Cells Explored,Number of Cell in Shortest Path to Goal
0,30X30,18.917969,0.074000,772,99


### Executing A* for Maze Size 40 X 40

In [16]:
astar_40 = A_Star_Search(40)

statistics = astar_40.execute_a_star_search()

statistics = statistics.style.applymap(lambda x:'white-space:nowrap')
display(statistics)


,Maze Size,Time Taken (in ms),Memory Consumed (in MB),Number of Cells Explored,Number of Cell in Shortest Path to Goal
0,40X40,30.916504,0.074000,1210,127
